In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-2
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -19.000000. running mean: -19.000000
episode 2.000000, reward total was -21.000000. running mean: -19.020000
episode 3.000000, reward total was -21.000000. running mean: -19.039800
episode 4.000000, reward total was -20.000000. running mean: -19.049402
episode 5.000000, reward total was -21.000000. running mean: -19.068908
episode 6.000000, reward total was -21.000000. running mean: -19.088219
episode 7.000000, reward total was -21.000000. running mean: -19.107337
episode 8.000000, reward total was -21.000000. running mean: -19.126263
episode 9.000000, reward total was -21.000000. running mean: -19.145001
episode 10.000000, reward total was -21.000000. running mean: -19.163551
episode 11.000000, reward total was -21.000000. running mean: -19.181915
episode 12.000000, reward total was -21.000000. running mean: -19.200096
episode 13.000000, reward total was -21.000000. running mean: -19.218095
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -20.000000. running mean: -20.101851
episode 115.000000, reward total was -20.000000. running mean: -20.100833
episode 116.000000, reward total was -21.000000. running mean: -20.109824
episode 117.000000, reward total was -21.000000. running mean: -20.118726
episode 118.000000, reward total was -20.000000. running mean: -20.117539
episode 119.000000, reward total was -21.000000. running mean: -20.126363
episode 120.000000, reward total was -20.000000. running mean: -20.125100
episode 121.000000, reward total was -21.000000. running mean: -20.133849
episode 122.000000, reward total was -21.000000. running mean: -20.142510
episode 123.000000, reward total was -21.000000. running mean: -20.151085
episode 124.000000, reward total was -21.000000. running mean: -20.159574
episode 125.000000, reward total was -21.000000. running mean: -20.167979
episode 126.000000, reward total was -21.000000. running mean: -20.176299
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.182362
episode 226.000000, reward total was -20.000000. running mean: -20.180538
episode 227.000000, reward total was -21.000000. running mean: -20.188733
episode 228.000000, reward total was -20.000000. running mean: -20.186846
episode 229.000000, reward total was -20.000000. running mean: -20.184977
episode 230.000000, reward total was -19.000000. running mean: -20.173127
episode 231.000000, reward total was -21.000000. running mean: -20.181396
episode 232.000000, reward total was -19.000000. running mean: -20.169582
episode 233.000000, reward total was -20.000000. running mean: -20.167886
episode 234.000000, reward total was -21.000000. running mean: -20.176207
episode 235.000000, reward total was -21.000000. running mean: -20.184445
episode 236.000000, reward total was -21.000000. running mean: -20.192601
episode 237.000000, reward total was -21.000000. running mean: -20.200675
episode 238.000000, reward total was -

episode 336.000000, reward total was -20.000000. running mean: -20.185232
episode 337.000000, reward total was -20.000000. running mean: -20.183379
episode 338.000000, reward total was -20.000000. running mean: -20.181546
episode 339.000000, reward total was -19.000000. running mean: -20.169730
episode 340.000000, reward total was -20.000000. running mean: -20.168033
episode 341.000000, reward total was -21.000000. running mean: -20.176352
episode 342.000000, reward total was -21.000000. running mean: -20.184589
episode 343.000000, reward total was -21.000000. running mean: -20.192743
episode 344.000000, reward total was -21.000000. running mean: -20.200816
episode 345.000000, reward total was -21.000000. running mean: -20.208807
episode 346.000000, reward total was -21.000000. running mean: -20.216719
episode 347.000000, reward total was -20.000000. running mean: -20.214552
episode 348.000000, reward total was -20.000000. running mean: -20.212407
episode 349.000000, reward total was -

...

episode 6518.000000, reward total was -14.000000. running mean: -7.503562
episode 6519.000000, reward total was -14.000000. running mean: -7.568526
episode 6520.000000, reward total was -9.000000. running mean: -7.582841
episode 6521.000000, reward total was -10.000000. running mean: -7.607012
episode 6522.000000, reward total was -3.000000. running mean: -7.560942
episode 6523.000000, reward total was -7.000000. running mean: -7.555333
episode 6524.000000, reward total was -8.000000. running mean: -7.559780
episode 6525.000000, reward total was 2.000000. running mean: -7.464182
episode 6526.000000, reward total was -2.000000. running mean: -7.409540
episode 6527.000000, reward total was -11.000000. running mean: -7.445445
episode 6528.000000, reward total was -15.000000. running mean: -7.520990
episode 6529.000000, reward total was -13.000000. running mean: -7.575780
episode 6530.000000, reward total was -9.000000. running mean: -7.590022
episode 6531.000000, reward total was -11.0000

episode 6630.000000, reward total was -5.000000. running mean: -7.945917
episode 6631.000000, reward total was -1.000000. running mean: -7.876458
episode 6632.000000, reward total was -9.000000. running mean: -7.887693
episode 6633.000000, reward total was -10.000000. running mean: -7.908816
episode 6634.000000, reward total was -7.000000. running mean: -7.899728
episode 6635.000000, reward total was -11.000000. running mean: -7.930731
episode 6636.000000, reward total was -11.000000. running mean: -7.961424
episode 6637.000000, reward total was -10.000000. running mean: -7.981809
episode 6638.000000, reward total was -4.000000. running mean: -7.941991
episode 6639.000000, reward total was -5.000000. running mean: -7.912571
episode 6640.000000, reward total was -3.000000. running mean: -7.863446
episode 6641.000000, reward total was -14.000000. running mean: -7.924811
episode 6642.000000, reward total was -13.000000. running mean: -7.975563
episode 6643.000000, reward total was -15.000

episode 6743.000000, reward total was -13.000000. running mean: -6.629602
episode 6744.000000, reward total was -11.000000. running mean: -6.673306
episode 6745.000000, reward total was -9.000000. running mean: -6.696573
episode 6746.000000, reward total was 10.000000. running mean: -6.529607
episode 6747.000000, reward total was -4.000000. running mean: -6.504311
episode 6748.000000, reward total was -9.000000. running mean: -6.529268
episode 6749.000000, reward total was 2.000000. running mean: -6.443976
episode 6750.000000, reward total was -1.000000. running mean: -6.389536
episode 6751.000000, reward total was -10.000000. running mean: -6.425640
episode 6752.000000, reward total was -5.000000. running mean: -6.411384
episode 6753.000000, reward total was -11.000000. running mean: -6.457270
episode 6754.000000, reward total was -8.000000. running mean: -6.472697
episode 6755.000000, reward total was -12.000000. running mean: -6.527970
episode 6756.000000, reward total was 2.000000.

episode 6855.000000, reward total was -12.000000. running mean: -6.365372
episode 6856.000000, reward total was -15.000000. running mean: -6.451718
episode 6857.000000, reward total was 5.000000. running mean: -6.337201
episode 6858.000000, reward total was -17.000000. running mean: -6.443829
episode 6859.000000, reward total was -15.000000. running mean: -6.529391
episode 6860.000000, reward total was -10.000000. running mean: -6.564097
episode 6861.000000, reward total was 9.000000. running mean: -6.408456
episode 6862.000000, reward total was -7.000000. running mean: -6.414371
episode 6863.000000, reward total was -3.000000. running mean: -6.380228
episode 6864.000000, reward total was -5.000000. running mean: -6.366425
episode 6865.000000, reward total was -7.000000. running mean: -6.372761
episode 6866.000000, reward total was 13.000000. running mean: -6.179034
episode 6867.000000, reward total was -9.000000. running mean: -6.207243
episode 6868.000000, reward total was -9.000000.

episode 6967.000000, reward total was -10.000000. running mean: -7.271538
episode 6968.000000, reward total was -10.000000. running mean: -7.298822
episode 6969.000000, reward total was -13.000000. running mean: -7.355834
episode 6970.000000, reward total was 2.000000. running mean: -7.262276
episode 6971.000000, reward total was -12.000000. running mean: -7.309653
episode 6972.000000, reward total was 3.000000. running mean: -7.206556
episode 6973.000000, reward total was -15.000000. running mean: -7.284491
episode 6974.000000, reward total was -15.000000. running mean: -7.361646
episode 6975.000000, reward total was -10.000000. running mean: -7.388029
episode 6976.000000, reward total was -9.000000. running mean: -7.404149
episode 6977.000000, reward total was -16.000000. running mean: -7.490108
episode 6978.000000, reward total was -10.000000. running mean: -7.515207
episode 6979.000000, reward total was -9.000000. running mean: -7.530055
episode 6980.000000, reward total was -7.000